In [ ]:
pip install pdfplumber

In [ ]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### Testing table extraction for the 1st page of the file

In [ ]:
file ='../data/raw_data/TX/TX_2013.pdf'

In [ ]:
pdf = pdfplumber.open(file)
table=pdf.pages[0].extract_table()
df_test = pd.DataFrame(table[1::],columns=table[0])

In [ ]:
df_test.head()

### Extracting all the tables from a PDF file

In [ ]:
#Create df from table on first page to act as the first df:
pdf = pdfplumber.open(file)
pages = pdf.pages
tbl = pages[0].extract_table()
original_df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])

#Append data from remaining tables/pages:
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for i,pg in enumerate(pages):
        df = pd.DataFrame(tbl[1::]) 
        tbl = pages[i].extract_table()
        df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes']) 
        original_df=original_df.merge(df, how='outer')      

In [ ]:
original_df.insert(0,'Year',2013)
original_df

### Importing poppulation data and merging both files

In [ ]:
pop_2010_2019 = pd.read_excel('../data/raw_data/TX/pop_est_2010_2019_tx.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

In [ ]:
pop_2020_2021 = pd.read_excel('../data/raw_data/TX/pop_est_2020_2021_tx.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimate Base',axis=1)
pop_2020_2021.head()

In [ ]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined['County'] = pop_combined['County'].str.strip()
pop_combined['County'] = pop_combined['County'].str.upper()
pop_combined.head()

### Merging pop df and carsh data for speciffic year

In [ ]:
var = 2013

In [ ]:
pdf = pdfplumber.open(file)
pages = pdf.pages
tbl = pages[0].extract_table()
original_df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for i,pg in enumerate(pages):
        df = pd.DataFrame(tbl[1::]) 
        tbl = pages[i].extract_table()
        df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes']) 
        original_df=original_df.merge(df, how='outer')      

In [ ]:
original_df['suspected_serious_injuries'] =original_df['suspected_serious_injuries'].str.replace(',', '').astype(float)
original_df['possible_injuries'] =original_df['possible_injuries'].str.replace(',', '').astype(float)
original_df['injuries'] = original_df['suspected_serious_injuries'] + original_df['possible_injuries']
df_combined = pd.merge(original_df,pop_combined[['County', var]],on="County",how='left')
df_combined=df_combined.drop(['fatal_crashes','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','possible_injuries'], axis=1)
df_combined.head()

In [ ]:
df_combined.info()

### Adding Metropolitans and Micropolitans

In [ ]:
metros = pd.read_excel('../data/raw_data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_tx = metros_df[metros_df['State Name']=='Texas']
metros_tx['County'] = metros_tx['County'].str.strip()
metros_tx['County'] = metros_tx['County'].str.upper()
metros_tx.head()

In [ ]:
df_combined_new = pd.merge(df_combined,metros_tx[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_new.head(20)

### Adding everything together to create combined reports by year

In [ ]:
for filename in os.listdir("../data/raw_data/TX/"):
    if filename.endswith(".pdf"):
       year = re.findall(r'\d{4}',filename)
       for var in year:
           name = var       
       pdf = pdfplumber.open(os.path.join("C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TX/", filename))
       tx_df = pd.DataFrame(columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
  'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])
       pages = pdf.pages
       for i,pg in enumerate(pages): 
        tbl = pages[i].extract_table()
        df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])
        tx_df = tx_df.append(df) 
        if i== 9:
            name = int(name)
            tx_df['suspected_serious_injuries'] = tx_df['suspected_serious_injuries'].str.replace(',', '').astype(float)
            tx_df['possible_injuries'] = tx_df['possible_injuries'].str.replace(',', '').astype(float)
            tx_df['non_incapacitating_injuries'] = tx_df['non_incapacitating_injuries'].str.replace(',', '').astype(float)
            tx_df['injuries'] = tx_df['suspected_serious_injuries'] + tx_df['possible_injuries'] + tx_df['non_incapacitating_injuries']
            tx_df = tx_df.drop(['fatal_crashes','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','possible_injury_crashes','non_injury_crashes','non_injuries','unknown_severity_crashes','unknown_injuries','possible_injuries'], axis=1)                    
            tx_df = tx_df[['County','fatalities','injuries','total_crashes']]
            tx_df['County'] = tx_df['County'].str.strip()
            tx_df['County'] = tx_df['County'].str.upper()
            df_combined_tx = pd.merge(tx_df,pop_combined[['County', name]],on="County",how='left') 
            df_combined_tx_new = pd.merge(df_combined_tx,metros_tx[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
            df_combined_tx_new.rename(columns = {name:'population'}, inplace = True)
            df_combined_tx_new.drop(index=df_combined_tx_new.index[-1],axis=0,inplace=True)
            df_combined_tx_new.insert(0,'Year',name)
            df_combined_tx_new.to_excel('../data/cleaned_data/%s_TX.xlsx' % name, engine='xlsxwriter',index=False)
        else:
            continue

### Combining all the files into a master file

In [ ]:
data_file_folder ='../data/cleaned_data/'
df = []
for file in os.listdir(data_file_folder):
        if file.endswith('_TX.xlsx'):             
             df.append(pd.read_excel(os.path.join(data_file_folder,file),sheet_name='Sheet1'))
df_tx_master = pd.concat(df, axis = 0)

In [ ]:
df_tx_master.isna().sum()

In [ ]:
df_tx_master.insert(1,'State','Texas') 
df_tx_master.head(10)

In [ ]:
df_tx_master.tail(10)

In [ ]:
df_tx_master.to_excel('../data/TX.xlsx', engine='xlsxwriter',index=False)